In [1]:
%load_ext autoreload
%autoreload 2

from dprot.datasets import LMDBDataset
import torch

In [2]:
train_ds = LMDBDataset(
    "train", seq_len=256, data_version="230412", toy=True
)


In [3]:
loader = torch.utils.data.DataLoader(train_ds, batch_size=1024, shuffle=False)
batch = next(iter(loader))

In [10]:
batch.shape

torch.Size([503, 256, 1024])

In [20]:
real, fake = batch[:256, :, :], batch[256:, :, :]
print(real.shape, fake.shape)

torch.Size([256, 256, 1024]) torch.Size([247, 256, 1024])


In [22]:
import k_diffusion as K
fid_score = K.evaluation.fid(real.mean(dim=1), fake.mean(dim=1))
print(fid_score)

tensor(683.9904)


In [24]:
kid_score = K.evaluation.kid(real.mean(dim=1), fake.mean(dim=1))
print(kid_score)

tensor(-20709376.)
